In [1]:
import numpy as np
import pandas as pd

import requests, bs4
from datetime import datetime
import math

from IPython.display import clear_output

In [2]:
dataset = pd.read_csv('data.csv')

In [3]:
dataset.head()

,URL,Label
0,https://nih.gov,0
1,http://www.perfumemart.com/,0
2,http://chronopost-service-enligne.net/56123s/r...,1
3,http://wvw.micloudappel.security-updates-myacc...,1
4,http://colach8x.beget.tech/likon/mqnger/drwx/P...,1


In [4]:
len(dataset)

8000

In [5]:
to_datetime = lambda x : datetime(int(x.getText()[:4]), int(x.getText()[5:7]), int(x.getText()[8:]))

In [6]:
webmetrics = pd.DataFrame()
def websiteInfo(website, label):
    url = 'https://www.whois.com/whois/'+website
    website_details = {'URL': [website],
                      'Label': [label]}
    res = requests.get(url)
    if res.status_code == requests.codes.ok:
        ressoup = bs4.BeautifulSoup(res.text, 'lxml')
        elems = ressoup.select('.df-value')
        try:
            regOn = to_datetime(elems[2].getText())
            expOn = to_datetime(elems[3].getText())
            duration = (expOn - regOn).total_seconds()//31556926
            website_details['domainRegistrationLength'] = [int(duration)]
        except Exception as e:
            website_details['domainRegistrationLength'] = [e]
    else:
        website_details['domainRegistrationLength'] = [-abs1]
        
    
    url = 'https://www.alexa.com/siteinfo/'+website
    res = requests.get(url)
    if res.status_code == requests.codes.ok:
        ressoup = bs4.BeautifulSoup(res.text, 'lxml')
        
        elems = ressoup.select('.num.purple')
        try:
            website_details['Search Traffic'] = [elems[0].getText()]
        except Exception as e:
            website_details['Search Traffic'] = [e]
        try:
            website_details['Bounce Rate'] = [elems[1].getText()]
        except Exception as e:
            website_details['Bounce Rate'] = [e]
        
        elems = ressoup.select('.big.data')
        try:
            website_details['Total Sites Linking in'] = [elems[1].getText()]
        except Exception as e:
            website_details['Total Sites Linking in'] = [e]
        
        elems = ressoup.select('.rankmini-rank')
        try:
            website_details['Alexa Rank'] = [elems[0].getText().strip()]
        except Exception as e:
            website_details['Alexa Rank'] = [e]
            
        elems = ressoup.select('.small.data')
        try:
            website_details['Daily Views per Visitor'] = [elems[1].getText().strip().split()[0]]
        except Exception as e:
            website_details['Daily Views per Visitor'] = [e]
        try:
            website_details['Daily Time on Site'] = [elems[2].getText().strip().split()[0]]
        except Exception as e:
            website_details['Daily Time on Site'] = [e]
    else:
        website_details['Search Traffic'] = [-1]
        website_details['Bounce Rate'] = [-1]
        website_details['Total Sites Linking in'] = [-1]
        website_details['Alexa Rank'] = [-1]
        website_details['Daily Views per Visitor'] = [-1]
        website_details['Daily Time on Site'] = [-1]
    
    print(website_details)
    website_details = pd.DataFrame(website_details)

    return website_details

In [7]:
i = 1
for x, y in zip(dataset['URL'], dataset['Label']):
    print(i)
    webmetrics = pd.concat([webmetrics, websiteInfo(x,y)], axis=0)
    print('-'*50)
    i+=1

1
{'URL': ['https://nih.gov'], 'Label': [0], 'domainRegistrationLength': [IndexError('list index out of range')], 'Search Traffic': [-1], 'Bounce Rate': [-1], 'Total Sites Linking in': [-1], 'Alexa Rank': [-1], 'Daily Views per Visitor': [-1], 'Daily Time on Site': [-1]}
--------------------------------------------------
2
{'URL': ['http://www.perfumemart.com/'], 'Label': [0], 'domainRegistrationLength': [IndexError('list index out of range')], 'Search Traffic': [-1], 'Bounce Rate': [-1], 'Total Sites Linking in': [-1], 'Alexa Rank': [-1], 'Daily Views per Visitor': [-1], 'Daily Time on Site': [-1]}
--------------------------------------------------
3
{'URL': ['http://chronopost-service-enligne.net/56123s/remboursement/remboursement/assurance/68e8bee20c2d73a1dcbf1dcd35ef13f3/'], 'Label': [1], 'domainRegistrationLength': [IndexError('list index out of range')], 'Search Traffic': [-1], 'Bounce Rate': [-1], 'Total Sites Linking in': [-1], 'Alexa Rank': [-1], 'Daily Views per Visitor': [-1

KeyboardInterrupt: 

In [ ]:
webmetrics

In [ ]:
webmetrics.to_csv('webmetrics.csv', index=False)